In [27]:
from flask import Flask, jsonify, request, render_template, send_file
import pandas as pd
from datetime import datetime, timedelta
import pytz
from pytz import timezone
import mysql.connector
from mysql.connector import Error
import csv, os, time

In [15]:
def get_connection():
    try:
        connection = mysql.connector.connect(
            host="localhost",
            port="8889",
            user="ABC",
            password="test123",
            database="Work_DB"
        )
        return connection
    except Error as e:
        print("Error while connecting to MySQL:", e)
        return None

In [16]:
def check_new_records():
    while(True):
        connection = get_connection()
        try:
            cursor=connection.cursor()

            #=====Menu hours.csv
            with open('Menu hours.csv','r') as file:
                csv_reader=csv.DictReader(file)
                
                cursor.execute('SELECT * from LK_menu_hours')
                existing_records = cursor.fetchall()

                '''
                Since records are of varying types, we convert them all to string using list comprehension.
                This enables us to compare rows from the csv with existing table records without worrying about their type. 
                '''
                
                existing_records = [tuple(str(value) for value in record) for record in existing_records]
                

                for row in csv_reader:
                    record = tuple(row.values())
                    
                    if record not in existing_records: # This also ensures no duplicate rows are in the tables
                        print("================ New record inserted in LK_menu_hours ================")
                        query = "INSERT INTO LK_menu_hours VALUES (%s, %s, %s, %s)"
                        cursor.execute(query, record)
                        connection.commit()
                        print(record)
                        
            #=====bq-results.csv
            with open('bq-results-20230125-202210-1674678181880.csv','r') as file:
                csv_reader=csv.DictReader(file)
                
                cursor.execute('SELECT * from LK_bq_result')
                existing_records = cursor.fetchall()
                
                existing_records = [tuple(str(value) for value in record) for record in existing_records]

                for row in csv_reader:
                    record = tuple(row.values())
                    if record not in existing_records: 
                        print("================ New record inserted in LK_bq_result ================")
                        query = "INSERT INTO LK_bq_result VALUES (%s, %s, %s, %s)"
                        cursor.execute(query, record)
                        connection.commit()
                        print(record)

            #=====Menu hours.csv
            with open('store status.csv','r') as file:
                csv_reader=csv.DictReader(file)
                
                cursor.execute('SELECT * from LK_store_status')
                existing_records = cursor.fetchall()
                
                existing_records = [tuple(str(value) for value in record) for record in existing_records]

                for row in csv_reader:
                    record = tuple(row.values())
                    
                    if record not in existing_records: 
                        print("================ New record inserted in LK_store_status ================")
                        query = "INSERT INTO LK_store_status VALUES (%s, %s, %s, %s)"
                        cursor.execute(query, record)
                        connection.commit()
                        print(record)

            cursor.close()
            connection.close()
        except Error as e:
            print(e)
        time.sleep(3600) #1 hour = 3600 sec

In [31]:
def get_stores(start_time, end_time):
    connection = get_connection()
    if connection:
        try:
            cursor = connection.cursor()
            query = """SELECT DISTINCT(store_id) FROM LK_store_status WHERE 
            timestamp_utc BETWEEN %s AND %s"""
            cursor.execute(query, (str(start_time), str(end_time)))
            stores=cursor.fetchall()
            
            #stores = [store[0] for store in cursor.fetchall()]
            cursor.close()
            connection.close()
            return stores
        except Error as e:
            print("Error executing query (in get_stores):", e)
            return None

In [18]:
def get_business_hours(store_id):
    connection = get_connection()
    if connection:
        try:
            cursor = connection.cursor()
            query = "SELECT day, start_time_local, end_time_local FROM LK_menu_hours WHERE store_id = %s"
            cursor.execute(query, (store_id,))
            result=cursor.fetchall()
            business_hours = {}
            cursor.close()
            connection.close()
            if result:
                
                for row in result:
                    day, start_time_local, end_time_local = row
                    business_hours[day] = (start_time_local, end_time_local)
            else:
                for i in range(0,6):
                    business_hours[i] = ('00:00:00', '23:59:59')
                    
            return business_hours
        except Error as e:
            print("Error executing query (in get_business_hours):", e)
            return None

In [19]:
def get_timezone(store_id):
    connection = get_connection()
    if connection:
        try:
            cursor = connection.cursor()
            query = "SELECT timezone_str FROM LK_bq_result WHERE store_id = %s"
            cursor.execute(query, (store_id,))
            result = cursor.fetchone()  #since we only have one timezone per store
            cursor.close()
            connection.close()
            return result[0] if result else 'America/Chicago' #deafult timestamp: america/chicago
        except Error as e:
            print("Error executing query (in get_timezone):", e)
            return None
            

In [20]:
def get_timestamp_in_business_hours(store_id, business_hours, timezn):
    connection = get_connection()
    if connection:
        try:
            cursor = connection.cursor()
            query = "SELECT timestamp_utc, status FROM LK_store_status WHERE store_id = %s ORDER BY timestamp_utc"
            cursor.execute(query, (str(store_id),))

            status_range=[]
            for row in cursor.fetchall():
                timestamp_utc = row[0]
                status=row[1]
                timestamp_local = pd.to_datetime(timestamp_utc).astimezone(pytz.timezone(timezn)) #convert from utc to local timezone
                timestamp_week=timestamp_local.weekday()

                for i in business_hours.keys():
                    if( timestamp_week == i and 
                    pd.to_datetime(business_hours[i][0]).time()<= timestamp_local.time()<= pd.to_datetime(business_hours[i][1]).time()):
                        status_range.append((timestamp_local, status))
            return status_range

        except Error as e:
            print("Error executing query (in get_timestamp_in_business_hours):", e)
            return None

In [21]:
def interpolate_status(store_id, max_timestamp, status_range):

    #====interpolated range for 1 week
     
    end_time = max_timestamp
    current_time = max_timestamp - timedelta(days=7)
    interpolated_time_1w=[]
    while ( current_time < end_time ): #not <= because if current_time = end_time will create duplicate
        for i in range ( 0, len(status_range)-1 ): #-1 since we are checking against i+1 items
            if(\
                status_range[i][0] <= current_time < status_range[i+1][0]
                ):
                interpolated_time_1w.append((current_time, status_range[i][1]))
                break
            else:
                interpolated_time_1w.append((status_range[i+1][0], status_range[i+1][1]))
                break
            
        current_time += timedelta(days=1)

    #====interpolated range for 1 day
    end_time = max_timestamp
    current_time = max_timestamp - timedelta(days=1)
    interpolated_time_1d=[]
    while ( current_time < end_time ): #not <= because if current_time = end_time will create duplicate
        for i in range ( 0, len(status_range)-1 ): #-1 since we are checking against i+1 items
            if(\
                status_range[i][0] <= current_time < status_range[i+1][0]
                ):
                interpolated_time_1d.append((current_time, status_range[i][1]))
                break
            else:
                interpolated_time_1d.append((status_range[i+1][0], status_range[i+1][1]))
                break
            
        current_time += timedelta(hours=1)

    #====interpolated range for 1 hour
    end_time = max_timestamp
    current_time = max_timestamp - timedelta(hours=1)
    interpolated_time_1h=[]
    while ( current_time < end_time ): #not <= because if current_time = end_time will create duplicate
        for i in range ( 0, len(status_range)-1 ): #-1 since we are checking against i+1 items
            if(\
                status_range[i][0] <= current_time < status_range[i+1][0]
                ):
                interpolated_time_1h.append((current_time, status_range[i][1]))
                break
            else:
                interpolated_time_1h.append((status_range[i+1][0], status_range[i+1][1]))
                break
            
        current_time += timedelta(minutes=1)


    return interpolated_time_1w, interpolated_time_1d, interpolated_time_1h
    

In [22]:
def get_uptime_downtime(interpolated_time_status):
    uptime = 0
    downtime = 0
    
    for current_time in interpolated_time_status:
        if ( current_time[1] == 'active' ):
            uptime += 1
        else:
            downtime += 1
    return uptime, downtime

In [23]:
def generate_report_id():
    report_id = os.urandom(8).hex()
    return report_id

In [24]:
#existing_report_id=[]

global report_id_lst
report_id_lst=['Generated Report IDs from Calling /trigger']

global generating_csv
generating_csv = True


In [25]:
def generate_report():
    connection = get_connection()
    if connection:
        try:
            cursor=connection.cursor()

            #set max timestamp as current time
            query = "SELECT MAX(timestamp_utc) FROM LK_store_status"
            cursor.execute(query)
            max_timestamp = cursor.fetchone()[0]
            
            max_timestamp=pd.to_datetime(max_timestamp)
            start_time = max_timestamp - timedelta(days=7)
            end_time = max_timestamp

            #get store records within start_time and end_time
            stores=get_stores(start_time, end_time)

            while(True):    #generate new report_id as long as there exists a duplicate
                #create csv file polled by report
                report_id=generate_report_id()

                if report_id not in report_id_lst: #checking if report_id is unique
                    report_id_lst.append(report_id)
                    csv_file = f'report_{report_id}.csv'
                    fieldnames = [
                        'store_id', 
                        'uptime_last_hour(minutes)', 'downtime_last_hour(minutes)',
                        'uptime_last_day(hours)', 'downtime_last_day(hours)',
                        'update_last_week(days)', 'downtime_last_week(days)'
                        ]
        
                    #======end for loop
                    global file
                    with open(csv_file, mode='w', newline='') as file:
                        writer=csv.DictWriter(file, fieldnames=fieldnames)
                        #add headers for the csv file
                        csv_headers={           
                            'store_id':'store_id',
                            'uptime_last_hour(minutes)':'uptime_last_hour(minutes)', 
                            'downtime_last_hour(minutes)':'downtime_last_hour(minutes)',
                            'uptime_last_day(hours)':'uptime_last_day(hours)', 
                            'downtime_last_day(hours)':'downtime_last_day(hours)',
                            'update_last_week(days)':'update_last_week(days)', 
                            'downtime_last_week(days)':'downtime_last_week(days)'
                            }
                        writer.writerow(csv_headers)

                         #=====For each store_id in store
                        for i in range(0, len(stores)):
                            store_id = stores[i][0]
                            #for testing
                            #store_id='5704365908807155452'

                            business_hours=get_business_hours(store_id)
                            timezn = get_timezone(store_id)
                            
                            status_range = get_timestamp_in_business_hours(store_id, business_hours, timezn)
                            #print(status_range[1])
                            
                            interpolated_time_status_1w, interpolated_time_status_1d, interpolated_time_status_1h = \
                                interpolate_status(store_id, max_timestamp, status_range)
                            #print(interpolated_time_status[0])

                            uptime_1w, downtime_1w = get_uptime_downtime(interpolated_time_status_1w)
                            uptime_1d, downtime_1d = get_uptime_downtime(interpolated_time_status_1d)
                            uptime_1h, downtime_1h = get_uptime_downtime(interpolated_time_status_1h)

                            record_write={
                                    'store_id': store_id, 
                                    'uptime_last_hour(minutes)':uptime_1h, 
                                    'downtime_last_hour(minutes)':downtime_1h,
                                    'uptime_last_day(hours)': uptime_1d, 
                                    'downtime_last_day(hours)':downtime_1d,
                                    'update_last_week(days)': uptime_1w, 
                                    'downtime_last_week(days)': downtime_1w
                                }
                            writer.writerow(record_write)
                        global generating_csv 
                        generating_csv=False 
                    break
                else:
                    print("Record already exists...regenarating new id...")
            
            
            cursor.close()
            connection.close()
            return(report_id)
        except Error as e:
            print("SQL query error: ",e)
            return None

In [ ]:
app=Flask(__name__, template_folder='template')

global generating_csv
generating_csv = True

@app.route('/')  #landing
def index():
    global report_id_lst
    return render_template('index.html', report_lst=report_id_lst)

@app.route('/csv_update')  #check new records every hour
def csv_update():
    check_new_records()
    return 'Check for new Records every hour'

@app.route('/trigger_report')
def trigger_report():       #trigger report generation
    global generating_csv
    generating_csv = True
    report_id = generate_report()
    return jsonify({'report_id': report_id})

@app.route('/get_report/<report_id>', methods=['GET', 'POST'])
def get_report(report_id):
    filename = f'report_{report_id}.csv'
    global generating_csv

    # all reports have generated
    if generating_csv == False and report_id in report_id_lst:
        file_df=pd.read_csv(filename, nrows=10)
        return render_template('status_complete.html', status='Complete', tables=file_df.to_html(), report_file=filename)
    
    #only last report is generating
    if generating_csv == True and report_id in report_id_lst[:-1]:
        file_df=pd.read_csv(filename, nrows=10)
        return render_template('status_complete.html', status='Complete', tables=file_df.to_html(), report_file=filename)
    
    #check if last report had generated
    if generating_csv==True:
            return "Running..."

@app.route('/download_csv/<filename>')
def download_csv(filename):
    #filename = f'report_{report_id}.csv'
    return send_file(filename, as_attachment=True)

if __name__=="__main__": 
    app.run()